In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.base import clone
import joblib
import pickle
import matplotlib.pyplot as plt
import doubleml as dml
import patsy
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot

from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
dates= pd.date_range('2017-01-01','2021-07-01' , freq='1M')-pd.offsets.MonthBegin(1)
dates=[date_obj.strftime('%Y-%m-%d') for date_obj in dates]

var_quel=["emploi_quelconque_t_plus_3","emploi_quelconque_t_plus_6",  "emploi_quelconque_t_plus_12" ,"emploi_quelconque_t_plus_18" ,"emploi_quelconque_t_plus_24","emploi_quelconque_t_plus_36"]

x_cols=["age", "anciennete","sexe","zone_urbaine","handicap" ]

path="C:/Users/dalil.youcefi/Documents/Formation et retour à l'emploi/Données"

In [3]:
li_poids=[]
for cohorte in dates:
    df=pd.read_parquet(path+"/Prepro/Toutes Form/prepro_data_"+cohorte+".parquet",engine='pyarrow')
    li_poids.append(df.shape[0])

In [4]:


dic_irm={}





dic_irm["cohorte"]=dates



for var in var_quel:
    irm_coef=[]

    irm_upp=[]
    irm_low=[]
  
    irm_std=[]
    
    
    for cohorte in dates:
     
      
        
        dml_irm_obj=joblib.load( path+"/Résultats/Python/IRM/ATT/dml_irm_att_"+cohorte+"_"+var+".joblib")
        irm_coef.append(dml_irm_obj.summary.coef)
        irm_low.append(dml_irm_obj.confint()["2.5 %"])
        irm_upp.append(dml_irm_obj.confint()["97.5 %"])
        irm_std.append(dml_irm_obj.summary["std err"])
        
        
   
    dic_irm["coef_"+var]=irm_coef
    dic_irm["upp_"+var]=irm_upp
    dic_irm["low_"+var]=irm_low
    dic_irm["std_"+var]=irm_std
    
    
 

   
df_irm=pd.DataFrame.from_dict(dic_irm,orient="columns")





In [21]:
dml_irm_obj.summary.coef

entree_formation    0.099528
Name: coef, dtype: float64

Reproduction résultats du matching et comparaison avec ATT IRM

In [29]:
df_irm["poids"]=li_poids
dic_avg={}



irm_coef=[]

irm_upp=[]
irm_low=[]

irm_std=[]



def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()
 


dic_avg={}


dic_avg["temps"]=["6","12","18","24"]




var="emploi_quelconque_t_plus_6"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]
    
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))


var="emploi_quelconque_t_plus_12"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period] 
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
var="emploi_quelconque_t_plus_18"
    

period=pd.date_range('2017-01-01','2019-04-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]   
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
var="emploi_quelconque_t_plus_24"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]    
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
    
    


dic_avg["irm_coef"]=irm_coef
dic_avg["irm_upp"]=irm_upp
dic_avg["irm_low"]=irm_low
dic_avg["irm_std"]=irm_std
df_avg=pd.DataFrame.from_dict(dic_avg,orient="columns")

df_avg=df_avg*100

df_avg["temps"]=["6","12","18","24"]    

In [30]:
df_avg=df_avg.astype(float)

Résultats matching valeurs

In [31]:
df_avg["match_coef"]=[0.6,5.7,8.8,9.2]
df_avg["match_std"]=[0.27,0.29,0.29,0.29]

In [32]:
np.round(df_avg,2)

,temps,irm_coef,irm_upp,irm_low,irm_std,match_coef,match_std
0,6.0,3.16,3.81,2.51,0.33,0.6,0.27
1,12.0,8.48,9.18,7.77,0.36,5.7,0.29
2,18.0,11.81,12.52,11.09,0.37,8.8,0.29
3,24.0,11.93,12.63,11.23,0.36,9.2,0.29
